In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# 準備

In [ ]:
%tensorflow_version 1.x
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Activation,Dropout,Flatten,Reshape
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Conv2D

# DCGANの基本構造の構築（Construction of basic structure of DCGAN）
- Gモデルに、入力値を2次元に変換する処理を適用
- Gモデルに、アップサンプリングを適用
- Gモデルに、畳み込み層を適用

- 


畳み込みの処理については[こちら](https://qiita.com/koshian2/items/885aba771190267a3a1c)

アップサンプリングについては[こちら](https://postd.cc/generating-faces/)

In [7]:
def build_generator():
    noise_shape = (z_dim, )
    activation = Activation("relu")

    model = Sequential()
　　　　
    model.add(Dense(1024, input_shape=noise_shape))
    model.add(activation)
    model.add(BatchNormalization())

    model.add(Dense(7*7*128))
    model.add(activation)
    model.add(BatchNormalization())
　　　　　　
　　　　　　# 畳み込み層に7 * 7の画像を与える
    model.add(Reshape((7,7,128),input_shape=(128*7*7,)))

　　　　　　# UpSamplingをし、画像を14 * 14にする
　　　　　　# 間の要素は０で埋める
    model.add(UpSampling2D((2,2)))
    model.add(Conv2D(64,5,padding="same"))
    model.add(activation)

    # UpSamplingをし、画像を28 * 28にする
    model.add(UpSampling2D((2,2)))

    # 畳み込む（ゼロパディングをして、5*5のレイヤーで畳み込み）
    model.add(Conv2D(1,5,padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    return model



def build_discriminator():
    activation = LeakyReLU(alpha=0.2)

    model = Sequential()

　　　　　　# 畳み込む（ゼロパディングをし、5*5のフィルタを2マスごとに動かす）
    model.add(Conv2D(64,5, strides=(2,2), padding="same", input_shape=(img_shape)))
    model.add(activation)

　　　　　　# 畳み込み2回目
    model.add(Conv2D(128,5, strides=(2,2)))
    model.add(activation)

    # 入力の平坦化（多分１次元化ってこと）
    model.add(Flatten())

    model.add(Dense(256))
    model.add(activation)
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation("sigmoid"))

    model.summary()

    return model


def build_combined():
    discriminator.trainable = False
    model = Sequential([generator, discriminator])
    model.summary()

    return model


#入力画像の定義
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols,channels)

#潜在変数の次元数
z_dim = 100

# 最適化関数
optimizer = Adam(lr = 0.0001, beta_1 = 0.5)

#discriminator
discriminator = build_discriminator()
discriminator.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics = ["accuracy"])

#generator
generator = build_generator()

combined_model = build_combined()
combined_model.compile(loss = "binary_crossentropy", optimizer = optimizer)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 128)         204928    
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               819456    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_

# 学習部分

In [0]:

def train(epochs, batch_size=128, save_interval=50):
        #mnist
        (X_train, _),(_, _) = mnist.load_data()

        #change -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5)/127.5

        #dim2 -> dim3
        X_train = np.expand_dims(X_train, axis = 3)

        half_batch = int(batch_size / 2)
        # num_batches = int(X_train.shape[0] / half_batch)
        num_batches = batch_size
        print("Number of Batches : ", num_batches)

        for epoch in range(epochs):
            for iteration in range(num_batches):
                #discriminator
                
                #バッチサイズの半分をgeneratorから作成
                noise = np.random.normal(0, 1, (half_batch, z_dim))
                gen_imgs = generator.predict(noise)

                #バッチサイズの半分を教師データからピックアップ
                idx = np.random.randint(0, X_train.shape[0], half_batch)
                imgs = X_train[idx]

                #training
                #本物と偽物は別々に
                d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
                d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

                #それぞれの損失関数を平均
                d_loss = np.add(d_loss_real, d_loss_fake) / 2




                #generator
                
                noise = np.random.normal(0, 1, (batch_size, z_dim))

                #正解に近づけるようにする(本物ラベル、1)
                valid_y = np.array([1] * batch_size)

                #training
                g_loss = combined_model.train_on_batch(noise, valid_y)

                # 進捗の表示
                print ("epoch:%d, iter:%d,  [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, iteration, d_loss[0], 100*d_loss[1], g_loss))

            # 指定した間隔で生成画像を保存
            if epoch % save_interval == 0:
                save_imgs(epoch)

# 生成画像の保存

In [0]:
def save_imgs(epoch):
    # 生成画像を敷き詰めるときの行数、列数
    r, c = 5, 5

    noise = np.random.normal(0, 1, (r * c, z_dim))
    gen_imgs = generator.predict(noise)

    # 生成画像を0-1に再スケール
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("drive/My Drive/GAN_images/dcgan_mnist_%d.png" % epoch)
    plt.close()

# 実行
- 学習速度が早い（全結合層をなくし、畳み込み層で抽出した特徴を、特徴マップごとの平均を取り、そのまま分類器にかけるため、調整するパラメータ数が少なくなる。ちょっとよくわからない）
- [ここ](https://elix-tech.github.io/ja/2017/02/06/gan.html)に説明あり
- ノイズがなく綺麗

In [13]:
import time

t1 = time.time() 
train(epochs=10, batch_size=100, save_interval=1)
t2 = time.time()

# 経過時間を表示
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

11493376/11490434 [==============================] - 1s 0us/step
Number of Batches :  100








/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0, iter:0,  [D loss: 0.722480, acc.: 20.00%] [G loss: 0.699979]
epoch:0, iter:1,  [D loss: 0.656297, acc.: 64.00%] [G loss: 0.685666]
epoch:0, iter:2,  [D loss: 0.617196, acc.: 64.00%] [G loss: 0.691398]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0, iter:3,  [D loss: 0.598628, acc.: 61.00%] [G loss: 0.675348]
epoch:0, iter:4,  [D loss: 0.595533, acc.: 53.00%] [G loss: 0.643932]
epoch:0, iter:5,  [D loss: 0.605774, acc.: 51.00%] [G loss: 0.594014]
epoch:0, iter:6,  [D loss: 0.644328, acc.: 50.00%] [G loss: 0.572222]
epoch:0, iter:7,  [D loss: 0.655207, acc.: 50.00%] [G loss: 0.580901]
epoch:0, iter:8,  [D loss: 0.648471, acc.: 50.00%] [G loss: 0.629736]
epoch:0, iter:9,  [D loss: 0.606433, acc.: 55.00%] [G loss: 0.730020]
epoch:0, iter:10,  [D loss: 0.590684, acc.: 63.00%] [G loss: 0.829354]
epoch:0, iter:11,  [D loss: 0.535994, acc.: 86.00%] [G loss: 0.935600]
epoch:0, iter:12,  [D loss: 0.539911, acc.: 86.00%] [G loss: 0.983959]
epoch:0, iter:13,  [D loss: 0.533309, acc.: 83.00%] [G loss: 0.949471]
epoch:0, iter:14,  [D loss: 0.604121, acc.: 72.00%] [G loss: 0.864981]
epoch:0, iter:15,  [D loss: 0.623863, acc.: 66.00%] [G loss: 0.721079]
epoch:0, iter:16,  [D loss: 0.653685, acc.: 59.00%] [G loss: 0.651918]
epoch:0, iter